In [50]:
# import libraries
import pandas as pd
import numpy as np
df = pd.read_excel("digitale wereld submap.xlsx")
df = df.drop(['@digitaal_watkanerbeter_extracategorie'], axis =1)
from sentence_transformers import SentenceTransformer, util

In [32]:
df.head()

,@digitaal_emoji,@digitaal_watgaatergoed_categorie,@digitaal_watgaatergoedd,@digitaal_watkanerbeter_categorie,@digitaal_watkanerbeterr
0,😐,['Verbinding en contact'],Veel mogelijkheden ook voor contact met mensen...,"['Mentale gezondheid ', 'Verslaving']",Ik vind dat het afgeschaft moet worden.
1,😐,"['Sociale media', 'Verbinding en contact', 'Sn...","in NL overal prima goed bereikbaar, we kunnen ...","['Sociale media', 'Veiligheid', 'Verslaving', ...",schadelijke kant en negatieve effecten van soc...
2,😡,"['Toegankelijkheid', 'Anders, namelijk...']",Dat er de laatste jaren meer restricties zijn ...,"['Verslaving', 'Leeftijd', 'Invloed', 'Mentale...","Heel veel, het internet neemt mensen nu over. ..."
3,😐,"['Toegang tot kennis en informatie', 'Verbindi...",Kennis die je snel hebt waardoor je veel kan l...,"['Invloed', 'Nepnieuws/ Desinformatie', 'Priva...",Het is teveel verslavend en er staan teveel nu...
4,😐,NaN,NaN,NaN,NaN


In [33]:
df = df.dropna(subset = ['@digitaal_watgaatergoed_categorie', '@digitaal_watgaatergoedd'])
df.head()

,@digitaal_emoji,@digitaal_watgaatergoed_categorie,@digitaal_watgaatergoedd,@digitaal_watkanerbeter_categorie,@digitaal_watkanerbeterr
0,😐,['Verbinding en contact'],Veel mogelijkheden ook voor contact met mensen...,"['Mentale gezondheid ', 'Verslaving']",Ik vind dat het afgeschaft moet worden.
1,😐,"['Sociale media', 'Verbinding en contact', 'Sn...","in NL overal prima goed bereikbaar, we kunnen ...","['Sociale media', 'Veiligheid', 'Verslaving', ...",schadelijke kant en negatieve effecten van soc...
2,😡,"['Toegankelijkheid', 'Anders, namelijk...']",Dat er de laatste jaren meer restricties zijn ...,"['Verslaving', 'Leeftijd', 'Invloed', 'Mentale...","Heel veel, het internet neemt mensen nu over. ..."
3,😐,"['Toegang tot kennis en informatie', 'Verbindi...",Kennis die je snel hebt waardoor je veel kan l...,"['Invloed', 'Nepnieuws/ Desinformatie', 'Priva...",Het is teveel verslavend en er staan teveel nu...
6,😡,"['Bewustzijn', 'Sociale media', 'Toegang tot k...","De mogelijkheid om te kunnen communiceren, spe...","['Bewustzijn', 'Privacy', 'Toegankelijkheid', ...",Er is veel media die bepaalde doelgroepen of m...


In [74]:
def string_to_list(s):
    return [str(x) for x in s.strip('[]').split(',')]

In [35]:
df['@digitaal_watgaatergoed_categorie'] = df['@digitaal_watgaatergoed_categorie'].apply(string_to_list)

In [48]:
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

In [36]:
def create_category_dict(df, sentence_column, category_column):
    category_dict = {}
    
    # Iterate over the DataFrame rows
    for _, row in df.iterrows():
        sentence = row[sentence_column]
        categories = row[category_column]
        
        # Add the sentence to each category in the dictionary
        for category in categories:
            if category not in category_dict:
                category_dict[category] = []
            category_dict[category].append(sentence)
    
    return category_dict

In [49]:
# Load the multilingual sentence transformer
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Predefined categories with example phrases (you can expand these later!)
category_examples = create_category_dict(train_df, "@digitaal_watgaatergoedd" , "@digitaal_watgaatergoed_categorie")

# Encode all category examples in advance
category_embeddings = {
    category: model.encode(examples, convert_to_tensor=True)
    for category, examples in category_examples.items()
}

In [38]:
def match_semantic_categories(text, threshold=0.6):
    # Encode input sentence
    input_embedding = model.encode(text, convert_to_tensor=True)
    
    matched = []
    for category, embeddings in category_embeddings.items():
        # Compute cosine similarities
        cosine_scores = util.cos_sim(input_embedding, embeddings)
        max_score = float(cosine_scores.max())  # Take the highest match
        
        if max_score >= threshold:
            matched.append((category, max_score))
    
    if matched:
        matched.sort(key=lambda x: x[1], reverse=True)
        return [cat for cat, _ in matched]
    else:
        return ["(Nieuwe categorie nodig)"]

In [47]:
# Example input sentence
sentence = "Door sociale media zoals Instagram blijven mensen in contact met elkaar."

# Match categories
categories = match_semantic_categories(sentence, threshold = 0.598)

# Output results
print("Zin:", sentence)
print("Herkennde categorieën:", categories)

Zin: Door sociale media zoals Instagram blijven mensen in contact met elkaar.
Herkennde categorieën: ["'Sociale media'", " 'Gebruiksvriendelijkheid'", " 'Veiligheid'", "'Privacy'", " 'Impact op dagelijks leven'"]


In [65]:
# Example scoring function
def custom_score(predicted, true):
    predicted_set = set(predicted)
    true_set = set(true)
    correct = len(predicted_set & true_set)
    extras = 0.2*len(predicted_set - true_set)
    return correct - extras


def evaluate_threshold(threshold, sentence_column, category_column ):
    scores = []
    for _, row in test_df.iterrows():
        predicted = match_semantic_categories(row[sentence_column], threshold=threshold)
        true = row[category_column]  # assume this is a list
        scores.append(custom_score(predicted, true))
    return np.mean(scores)

In [66]:
thresholds = np.linspace(0.1, 0.9, 100)
results = []

best_score = -np.inf
best_threshold = None

for t in thresholds:
    score = evaluate_threshold(t, "@digitaal_watgaatergoedd" , "@digitaal_watgaatergoed_categorie")
    results.append((t, score))
    print(f"Threshold {t:.3f} -> Score: {score:.4f}")
    
    if score > best_score:
        best_score = score
        best_threshold = t

print(f"\n🎯 Best threshold: {best_threshold:.3f} with custom score: {best_score:.4f}")

Threshold 0.100 -> Score: -1.0444
Threshold 0.108 -> Score: -0.9889
Threshold 0.116 -> Score: -0.9667
Threshold 0.124 -> Score: -0.9333
Threshold 0.132 -> Score: -0.8556
Threshold 0.140 -> Score: -0.8111
Threshold 0.148 -> Score: -0.8111
Threshold 0.157 -> Score: -0.8667
Threshold 0.165 -> Score: -0.9333
Threshold 0.173 -> Score: -0.8778
Threshold 0.181 -> Score: -0.7667
Threshold 0.189 -> Score: -0.7444
Threshold 0.197 -> Score: -0.7111
Threshold 0.205 -> Score: -0.7000
Threshold 0.213 -> Score: -0.6889
Threshold 0.221 -> Score: -0.6556
Threshold 0.229 -> Score: -0.6444
Threshold 0.237 -> Score: -0.5333
Threshold 0.245 -> Score: -0.4778
Threshold 0.254 -> Score: -0.3333
Threshold 0.262 -> Score: -0.3222
Threshold 0.270 -> Score: -0.2778
Threshold 0.278 -> Score: -0.1778
Threshold 0.286 -> Score: -0.2667
Threshold 0.294 -> Score: -0.3667
Threshold 0.302 -> Score: -0.4000
Threshold 0.310 -> Score: -0.2222
Threshold 0.318 -> Score: -0.2000
Threshold 0.326 -> Score: -0.1222
Threshold 0.33

KeyboardInterrupt: 

In [68]:
sentence_column, category_column=  "@digitaal_watgaatergoedd" , "@digitaal_watgaatergoed_categorie"
for _, row in test_df.iterrows():
        predicted = match_semantic_categories(row[sentence_column], threshold=0.43)
        true = row[category_column]  # assume this is a list
        print(row[sentence_column])
        print(predicted, true)
        print(custom_score(predicted, true))

in NL overal prima goed bereikbaar, we kunnen werken overal vandaan, ik kan eenvoudig met iedereen in contact blijven
["'Verbinding en contact'", " 'Plezier'", " 'Verbinding en contact'", "'Sociale media'"] ["'Sociale media'", " 'Verbinding en contact'", " 'Snelle vooruitgang'"]
1.6
Dat er de laatste jaren meer restricties zijn voor jonge mensen. _x000D_

['(Nieuwe categorie nodig)'] ["'Toegankelijkheid'", " 'Anders", " namelijk...'"]
-0.2
Toegankelijkheid.
["'Toegang tot kennis en informatie'"] ["'Gebruiksvriendelijkheid'", " 'Toegankelijkheid'", " 'Mediawijsheid'", " 'Verbinding en contact'", " 'Toegang tot kennis en informatie'"]
-0.2
Ik ben tevreden over sociale media 
["'Bewustzijn'", " 'Impact op dagelijks leven'", " 'Verbinding en contact'", " 'Plezier'", " 'Privacy'"] ["'Sociale media'", " 'Veiligheid'", " 'Gebruiksvriendelijkheid'", " 'Bewustzijn'", " 'Privacy'", " 'Toegankelijkheid'", " 'Mediawijsheid'", " 'Toegang tot kennis en informatie'", " 'Snelle vooruitgang'", " 'Plezi

In [76]:
df = pd.read_excel("digitale wereld submap.xlsx")
df = df.drop(['@digitaal_watkanerbeter_extracategorie'], axis =1)


sentence_column, category_column=  "@digitaal_watkanerbeterr" , "@digitaal_watkanerbeter_categorie"
df = df.dropna(subset = [category_column, sentence_column])
df[category_column] = df[category_column].apply(string_to_list)

train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

# Load the multilingual sentence transformer
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Predefined categories with example phrases (you can expand these later!)
category_examples = create_category_dict(train_df, sentence_column , category_column)

# Encode all category examples in advance
category_embeddings = {
    category: model.encode(examples, convert_to_tensor=True)
    for category, examples in category_examples.items()
}

for _, row in test_df.iterrows():
        predicted = match_semantic_categories(row[sentence_column], threshold=0.43)
        true = row[category_column]  # assume this is a list
        print(row[sentence_column])
        print(predicted, true)
        print(custom_score(predicted, true))

schadelijke kant en negatieve effecten van sociale media, niet iedereen kan digitaal even goed mee, 
["'Privacy'", " namelijk...'", " 'Eenzaamheid'", " 'Mentale gezondheid '", " 'Verslaving'", " 'Anders", " 'Prestatiedruk'", " 'Sociale media'", "'Nepnieuws/ Desinformatie'"] ["'Sociale media'", " 'Veiligheid'", " 'Verslaving'", " 'Nepnieuws/ Desinformatie'", " 'Cyberpesten '"]
-0.6000000000000001
Heel veel, het internet neemt mensen nu over. Mensen kunnen er niet meer zonder en worden er enorm door beïnvloed. Er wordt wel gezegd dat mensen zo fantastisch zijn en zoveel kunnen, maar die zelfde mensen zijn verslaafd aan hulpmiddelen van het internet en zijn zelf steeds minder creatief._x000D_
_x000D_
Wij hechten TE veel waarde aan het internet. Dit moet echt veranderen, want met de generatie worden kinderen minder intelligent. 
["'Bewustzijn'", " 'Veiligheid'", " 'Sociale media'", " 'Mentale gezondheid '", " 'Verslaving'", " 'Nepnieuws/ Desinformatie'", " 'Mediawijsheid'", " 'Privacy'", "